In [4]:
from tqdm import tqdm

from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.recharge.esp import ESPSettings
#from openff.recharge.esp.psi4 import Psi4ESPGenerator
#from optimize.openff_psi4_gen import Psi4ESPGenerator
from openff.recharge.esp.storage import MoleculeESPRecord, MoleculeESPStore
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.utilities.molecule import smiles_to_molecule
from openff.recharge.grids import GridSettingsType, GridGenerator

from openff.units.elements import SYMBOLS
from rdkit.Chem import AllChem
from openff.toolkit import Molecule
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

import os
import numpy as np
import random

In [2]:
#load our molecule to create the conformers

molecule = smiles_to_molecule("OCC(O)CO")
rdmol = molecule.to_rdkit()
AllChem.EmbedMultipleConfs(rdmol, numConfs=10, randomSeed=42)

In [3]:
#test the optimizer

optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
optimize_MMFF



0

In [4]:
#test all the conformers

for id in range(rdmol.GetNumConformers()):
    optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94',confId=3) #,confId=1,mmffvariant='MMFF94'
    print(f'conformer:{id}, optimize (0 = converge, 1 = more iterations req): {optimize_MMFF}')

conformer:0, optimize (0 = converge, 1 = more iterations req): 0
conformer:1, optimize (0 = converge, 1 = more iterations req): 0
conformer:2, optimize (0 = converge, 1 = more iterations req): 0
conformer:3, optimize (0 = converge, 1 = more iterations req): 0
conformer:4, optimize (0 = converge, 1 = more iterations req): 0
conformer:5, optimize (0 = converge, 1 = more iterations req): 0
conformer:6, optimize (0 = converge, 1 = more iterations req): 0
conformer:7, optimize (0 = converge, 1 = more iterations req): 0
conformer:8, optimize (0 = converge, 1 = more iterations req): 0
conformer:9, optimize (0 = converge, 1 = more iterations req): 0


In [5]:
rdmol.GetConformer(1)

In [6]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol


In [7]:
#do the conformers look structurally sensible

for id in range(rdmol.GetNumConformers()):
    IPythonConsole.drawMol3D(rdmol,confId=id)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
optimize_MMFF

0

In [9]:
#now create a series of xyz files so we can run the conformers with ASE, first convert our conformers to xyz format and then write to file

from openff.units import unit

cwd = os.getcwd()

conformers = []



for confs in range(rdmol.GetNumConformers()):
    conformer = np.zeros((rdmol.GetConformer(confs).GetNumAtoms(), 3))
    for atom_index, coordinates in enumerate(rdmol.GetConformer(confs).GetPositions()):
        conformer[atom_index, :] = coordinates
    conformers.append(conformer * unit.angstrom)


for number ,conformer in enumerate(conformers):
    atoms = [
            {
                "element": SYMBOLS[atom.atomic_number],
                "x": conformer[index, 0],
                "y": conformer[index, 1],
                "z": conformer[index, 2],
            }
            for index, atom in enumerate(molecule.atoms)
        ]
    xyz = f'{molecule.n_atoms}\n{molecule.to_smiles()}\n'
    for row in atoms:
        xyz += f"{row['element']}\t{np.around(row['x'].magnitude,decimals=6)}\t{np.around(row['y'].magnitude,decimals=6)}\t{np.around(row['z'].magnitude, decimals=6)}\n"
    try:
        f = open(cwd+f"/conformer_{number}.xyz", 'x')
        f.write(xyz)
        f.close()
    except FileExistsError:
        continue

In [10]:
from ase.calculators.gaussian import Gaussian, GaussianOptimizer
import ase.io
from ase.visualize import view
!module load gaussian/09




/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [11]:
Gaussian.command = '/mnt/storage/apps/eb/software/gaussian/09/g09 < PREFIX.com > PREFIX.log'

In [12]:
test_atoms = ase.io.read('conformer_0.xyz')

In [13]:
view(test_atoms)

<Popen: returncode: None args: ['/mnt/nfs/home/nca121/mambaforge/envs/openff...>

In [2]:
from openff.recharge.utilities.molecule import smiles_to_molecule
from openff.recharge.conformers import ConformerGenerator, ConformerSettings
from openff.units.elements import SYMBOLS
import numpy as np
from openff.units import unit


In [15]:
molecule = smiles_to_molecule("OCC(O)CO")
conformers = ConformerGenerator.generate(
    molecule, ConformerSettings(max_conformers=10)
)

In [16]:
def conf_to_xyz_string(conformer, molecule):
    atoms = [
                    {
                        "element": SYMBOLS[atom.atomic_number],
                        "x": conformer[index, 0],
                        "y": conformer[index, 1],
                        "z": conformer[index, 2],
                    }
                    for index, atom in enumerate(molecule.atoms)
                ]
    xyz = f'{molecule.n_atoms}\n{molecule.to_smiles()}\n'
    for row in atoms:
        xyz += f"{row['element']}\t{np.around(row['x'].magnitude,decimals=6)}\t{np.around(row['y'].magnitude,decimals=6)}\t{np.around(row['z'].magnitude, decimals=6)}\n"
    return xyz

In [17]:
conformers[0]

Magnitude,[[-0.3611847460269928 -0.008062569424510002 0.021483348682522774] [0.9523459672927856 0.5384629368782043 -0.032555222511291504] [1.1439802646636963 1.233094573020935 -1.3743836879730225] [2.4670777320861816 1.7732906341552734 -1.4064496755599976] [0.9855600595474243 0.2572946548461914 -2.534839153289795] [1.1710933446884155 0.9411581158638 -3.7698163986206055] [-0.45401597023010254 -0.4434690475463867 0.8860717415809631] [1.0573564767837524 1.2441487312316895 0.7981725335121155] [1.6660577058792114 -0.28194597363471985 0.09198283404111862] [0.4551912844181061 2.0798325538635254 -1.4719767570495605] [2.5237202644348145 2.3979244232177734 -0.6684095859527588] [-0.009874296374619007 -0.19747895002365112 -2.545755624771118] [1.7366387844085693 -0.5375308394432068 -2.487229347229004] [1.0608800649642944 0.2808915376663208 -4.472022533416748]]
Units,angstrom


In [18]:
type(conformers[0])

openff.units.units.Quantity

In [19]:
from openff.units import unit

cwd = os.getcwd()

conformers = []

molecule = smiles_to_molecule("OCC(O)CO")
rdmol = molecule.to_rdkit()
AllChem.EmbedMultipleConfs(rdmol, numConfs=10, randomSeed=42)

for confs in range(rdmol.GetNumConformers()):
    
    conformer = np.zeros((rdmol.GetConformer(confs).GetNumAtoms(), 3))
    for atom_index, coordinates in enumerate(rdmol.GetConformer(confs).GetPositions()):
        conformer[atom_index, :] = coordinates
    conformers.append(conformer * unit.angstrom)


In [20]:
[molecule.add_conformer(i) for i in conformers]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [21]:
rdmol =  molecule.to_rdkit()

In [22]:
rdmol.GetNumConformers()

10

In [23]:
optimize_MMFF = MMFFOptimizeMolecule(rdmol,mmffVariant = 'MMFF94')

In [24]:
optimized_conformer = Molecule.from_rdkit(rdmol)

In [25]:
optimized_conformer.n_conformers


10

In [2]:
from openff.toolkit import Molecule
import qcengine
from qcelemental.models import AtomicInput, OptimizationInput
from qcelemental.models.common_models import Model
from qcelemental.models.procedures import QCInputSpecification
from openff.recharge.utilities.molecule import smiles_to_molecule
from conformer_gen import Conformers


ModuleNotFoundError: No module named 'conformer_gen'

In [ ]:
# make a off toolkit molecule
molecule = Molecule.from_smiles("OCC(O)CO")
# Define the grid that the electrostatic properties will be trained on and the
# level of theory to compute the properties at.
grid_settings = LatticeGridSettings(
    type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
)
# Generating an MSK style grid is also supported:
# from openff.recharge.grids import MSKGridSettings
# grid_settings = MSKGridSettings(density=1.0)

esp_settings = ESPSettings(basis="6-31G*", method="hf", grid_settings=grid_settings)

# Generate a set of conformers for the molecule. We will compute the ESP and
# electric field for the molecule in each conformer.
#conformers = ConformerGenerator.generate(
#    molecule, ConformerSettings(max_conformers=10)
#)
conformers = Conformers.generate(molecule, generation_type='rdkit')
molecule.n_conformers

10

In [ ]:
qcel_mol = molecule.to_qcschema(conformer = 9)
qcel_mol.dict()


{'schema_name': 'qcschema_molecule',
 'schema_version': 2,
 'validated': True,
 'symbols': array(['O', 'C', 'C', 'O', 'C', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
        'H'], dtype='<U1'),
 'geometry': array([[-0.68254025, -0.01523605,  0.04059765],
        [ 1.79967305,  1.01754748, -0.06152045],
        [ 2.16180939,  2.33021103, -2.59720876],
        [ 4.66210124,  3.35103364, -2.65780469],
        [ 1.86243859,  0.48621643, -4.79015177],
        [ 2.21304569,  1.77853108, -7.12392053],
        [-1.8578508 ,  1.38506063, -0.14649441],
        [ 1.99811416,  2.35110036,  1.50832749],
        [ 3.14839277, -0.53280067,  0.17382236],
        [ 0.86018686,  3.93031391, -2.78163293],
        [ 4.76914011,  4.53142043, -1.26311106],
        [-0.01865972, -0.37318113, -4.81078091],
        [ 3.28177168, -1.01578607, -4.70018228],
        [ 2.00477277,  0.53080808, -8.45089781]]),
 'name': 'C3H8O3',
 'molecular_charge': 0.0,
 'molecular_multiplicity': 1,
 'connectivity': [(0, 1, 1.0),
  (

In [ ]:
xtb_model = Model(method="gfn2-xtb", basis=None)
qc_task = AtomicInput(molecule=qcel_mol, driver="energy", model=xtb_model)
qc_task


AtomicInput(driver='energy', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='f8a7a08')

In [ ]:
result = qcengine.compute(input_data=qc_task, program="xtb")
result.dict()


{'id': None,
 'schema_name': 'qcschema_output',
 'schema_version': 1,
 'molecule': {'schema_name': 'qcschema_molecule',
  'schema_version': 2,
  'validated': True,
  'symbols': array(['O', 'C', 'C', 'O', 'C', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
         'H'], dtype='<U1'),
  'geometry': array([[-0.68254025, -0.01523605,  0.04059765],
         [ 1.79967305,  1.01754748, -0.06152045],
         [ 2.16180939,  2.33021103, -2.59720876],
         [ 4.66210124,  3.35103364, -2.65780469],
         [ 1.86243859,  0.48621643, -4.79015177],
         [ 2.21304569,  1.77853108, -7.12392053],
         [-1.8578508 ,  1.38506063, -0.14649441],
         [ 1.99811416,  2.35110036,  1.50832749],
         [ 3.14839277, -0.53280067,  0.17382236],
         [ 0.86018686,  3.93031391, -2.78163293],
         [ 4.76914011,  4.53142043, -1.26311106],
         [-0.01865972, -0.37318113, -4.81078091],
         [ 3.28177168, -1.01578607, -4.70018228],
         [ 2.00477277,  0.53080808, -8.45089781]]),
  'name'

In [ ]:
# now perform an optimization using geometric
# make sure to change the model and program here
geometric_input = OptimizationInput(initial_molecule=qcel_mol, input_specification=QCInputSpecification(model=xtb_model), keywords={"coordsys": "tric", "maxiter": 300, "program": "xtb"})
geometric_input

OptimizationInput(model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='f8a7a08')

In [ ]:
opt_result = qcengine.compute_procedure(input_data=geometric_input, procedure="geometric")


In [ ]:
opt_result.initial_molecule

NGLWidget()

In [ ]:
opt_result.final_molecule

NGLWidget()

In [ ]:
opt_result.trajectory

[AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='f8a7a08'),
 AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='604ca64'),
 AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='e464bfd'),
 AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='ea08e05'),
 AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='84fa647'),
 AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='07f8e2c'),
 AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='bb578ff'),
 AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='cff644b'),
 AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='b6cf68e'),
 AtomicResult(driver='gradient', model={'method': 'gfn2-xtb', 'b

In [ ]:
qcel_mol

NGLWidget()

In [ ]:
ff_optimised_mol = opt_result.final_molecule
ff_optimised_mol.dict()

{'schema_name': 'qcschema_molecule',
 'schema_version': 2,
 'validated': True,
 'symbols': array(['O', 'C', 'C', 'O', 'C', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
        'H'], dtype='<U1'),
 'geometry': array([[-0.9189487935123769,  0.1565045522753112, -0.1284278242342739],
        [ 1.568705985128284 ,  1.0672785945918142, -0.0635727388756277],
        [ 2.3931424378368202,  2.2942444834530566, -2.571126565539323 ],
        [ 4.934320564733535 ,  3.0803552511810572, -2.438556976359111 ],
        [ 2.1356084997255707,  0.4456702699927401, -4.7834450061300675],
        [ 1.1733144276833802,  1.7717109885122937, -6.887045988130065 ],
        [-2.026968724972794 ,  1.5000210009196244, -0.6662112279139575],
        [ 1.8105065626598256,  2.4469685980066127,  1.4778104267920942],
        [ 2.7707952038907586, -0.5651055211990532,  0.3281384712479404],
        [ 1.1503908778435343,  3.9154096454595733, -2.983026004467564 ],
        [ 5.103065732234562 ,  4.376249664275655 , -1.1710895360615

In [ ]:
psi4_model = Model(method="HF", basis="6-31G*")


In [ ]:
qc_task = AtomicInput(molecule=ff_optimised_mol, driver="energy", model=xtb_model)


In [ ]:
qcel_mol = molecule.to_qcschema(conformer = 9)

opt_input = {
    "keywords": {
        "program": "xtb"
    },
    "input_specification": {
        "driver": "gradient",
        "model": {"method": "gfn2-xtb"},
    },
    "initial_molecule": qcel_mol
}

opt = qcengine.compute_procedure(opt_input, "geometric")

In [ ]:
ff_optimised_mol_2 = opt.final_molecule

In [ ]:
ff_optimised_mol_2.dict()

{'schema_name': 'qcschema_molecule',
 'schema_version': 2,
 'validated': True,
 'symbols': array(['O', 'C', 'C', 'O', 'C', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H',
        'H'], dtype='<U1'),
 'geometry': array([[-0.9189487935123809,  0.1565045522755265, -0.1284278242339979],
        [ 1.5687059851283514,  1.0672785945918508, -0.0635727388755923],
        [ 2.393142437836719 ,  2.2942444834530655, -2.571126565539358 ],
        [ 4.934320564733405 ,  3.080355251181167 , -2.4385569763592847],
        [ 2.1356084997254436,  0.4456702699926566, -4.783445006130014 ],
        [ 1.1733144276839735,  1.7717109885122866, -6.887045988130283 ],
        [-2.0269687249727597,  1.5000210009199237, -0.6662112279135486],
        [ 1.81050656266015  ,  2.4469685980066096,  1.4778104267921233],
        [ 2.7707952038907497, -0.5651055211991046,  0.3281384712478351],
        [ 1.1503908778433485,  3.91540964545952  , -2.9830260044675905],
        [ 5.103065732234388 ,  4.37624966427596  , -1.1710895360619

In [ ]:
ff_optimised_mol.dict()['geometry'] -  ff_optimised_mol_2.dict()['geometry']

array([[ 3.9968028886505635e-15, -2.1527224447481785e-13,
        -2.7600144392181392e-13],
       [-6.7279515292284486e-14, -3.6637359812630166e-14,
        -3.5374481122119050e-14],
       [ 1.0125233984581428e-13, -8.8817841970012523e-15,
         3.5083047578154947e-14],
       [ 1.2967404927621828e-13, -1.0969003483296547e-13,
         1.7363888105137448e-13],
       [ 1.2700951401711791e-13,  8.3488771451811772e-14,
        -5.3290705182007514e-14],
       [-5.9330318435968366e-13,  7.1054273576010019e-15,
         2.1760371282653068e-13],
       [-3.4194869158454821e-14, -2.9931612743894220e-13,
        -4.0889513996944515e-13],
       [-3.2440716779547074e-13,  3.1086244689504383e-15,
        -2.9087843245179101e-14],
       [ 8.8817841970012523e-15,  5.1403326040144748e-14,
         1.0530465388569610e-13],
       [ 1.8585133432225120e-13,  5.3290705182007514e-14,
         2.6645352591003757e-14],
       [ 1.7408297026122455e-13, -3.0464519795714295e-13,
         3.67483821150

In [ ]:
opt_input = {
    "keywords": {
        "program": "psi4"
    },
    "input_specification": {
        "driver": "gradient",
        "model": {"method": "hf", "basis": "6-31g"},
    },
    "initial_molecule": qcel_mol
}

opt = qcengine.compute_procedure(opt_input, "geometric")

In [ ]:
opt

OptimizationResult(model={'method': 'hf', 'basis': '6-31g'}, molecule_hash='f8a7a08')

In [ ]:
qcel_mol = molecule.to_qcschema(conformer = 9)

opt_input = {
    "keywords": {
        "program": "xtb",
        "verbosity" : "muted",

    },
    "input_specification": {
        "driver": "gradient",
        "model": {"method": "gfn2-xtb"},
    },
    "initial_molecule": qcel_mol
}

opt = qcengine.compute_procedure(opt_input, "geometric")


OptimizationResult(model={'method': 'gfn2-xtb', 'basis': None}, molecule_hash='f8a7a08')

In [ ]:
Molecule.from_qcschema(opt.final_molecule)

NGLWidget()

In [ ]:
Molecule.from_qcschema(opt.final_molecule).conformers

[array([[-0.4862867595144359,  0.0828186424668756, -0.0679610778306531],
        [ 0.8301234579386073,  0.564779509943595 , -0.0336412446477216],
        [ 1.2663964405504002,  1.2140618968855181, -1.3605815848332306],
        [ 2.61112999415179  ,  1.630053800413438 , -1.2904287793802247],
        [ 1.1301153494675436,  0.2358385504575636, -2.5312900868564645],
        [ 0.6208912563550152,  0.9375490794288812, -3.6444677873661515],
        [-1.072625656470663 ,  0.7937769295641217, -0.3525437994602623],
        [ 0.9580788131518267,  1.294880017862708 ,  0.7820235998946765],
        [ 1.4662416779810485, -0.2990409635746084,  0.1736434010052163],
        [ 0.6087606361865544,  2.071945555730803 , -1.5785493810981646],
        [ 2.700426091243515 ,  2.3158115915609283, -0.6197138944120185],
        [ 0.4469033281134626, -0.5762756189603794, -2.2470681640843884],
        [ 2.1225064336575907, -0.1794231142025792, -2.7403420164626615],
        [ 0.6630495280475959,  0.3672464765611082, 

In [ ]:
   grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )

In [ ]:
grid_settings

LatticeGridSettings(type='fcc', spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0)

In [6]:
import sys
print(sys.path)
sys.path.append('/mnt/storage/nobackup/nca121/test_jobs/QM_ESP_Psi4/QM_ESP_Psi4/')
from qcelemental.models.common_models import Model
from source.conformers.conformer_gen import Conformers
from qcelemental.models.procedures import OptimizationInput, QCInputSpecification
import qcengine
import qcelemental
from openff.units import unit

['/mnt/storage/nobackup/nca121/test_jobs/QM_ESP_Psi4/QM_ESP_Psi4/examples', '/home/user/psiadditions', '/mnt/storage/nobackup/nca121/test_jobs/QM_ESP_Psi4/QM_ESP_Psi4/examples', '/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python311.zip', '/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11', '/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/lib-dynload', '', '/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages']


In [4]:

test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)


hf_model = Model(method="hf", basis="6-31G*")
spec = QCInputSpecification(model=hf_model, keywords={}, driver="gradient")
opt_spec = OptimizationInput(
            initial_molecule=qc_mol,
            input_specification=spec,
            keywords={"coordsys": "dlc", 
                      "program": "psi4"}                                        
        )


opt = qcengine.compute_procedure(opt_spec, "geometric")
print(f'geometry optimiztion was {opt.error}')
optmized_mol = opt.final_molecule

opt_input = { "molecule" : optmized_mol,
              "driver" : "energy",
              "model" : {"method":"hf","basis":"6-31G*"},
              "protocols":{"wavefunction":"all"}

}

opt_2 = qcengine.compute(opt_input, "psi4")


rdkit has generated 10 conformers from a requested 10 conformers


geometry optimiztion was None


In [15]:
optmized_mol.geometry*unit.angstrom


Magnitude,[[-1.42427673006767 -0.36399129897769666 0.0] [-0.0014279097249317804 0.7223600294089259 0.0] [1.4257046397926019 -0.35836872043122914 0.0]]
Units,angstrom


In [5]:
print(opt_2.dict().keys())

opt_2.error
opt_2.wavefunction

NGLWidget()

In [5]:
opt_2.dict()

{'id': None,
 'schema_name': 'qcschema_output',
 'schema_version': 1,
 'molecule': {'schema_name': 'qcschema_molecule',
  'schema_version': 2,
  'validated': True,
  'symbols': array(['H', 'O', 'H'], dtype='<U1'),
  'geometry': array([[-1.42427673, -0.3639913 ,  0.        ],
         [-0.00142791,  0.72236003,  0.        ],
         [ 1.42570464, -0.35836872,  0.        ]]),
  'name': 'H2O',
  'molecular_charge': 0.0,
  'molecular_multiplicity': 1,
  'connectivity': [(0, 1, 1.0), (1, 2, 1.0)],
  'fix_com': True,
  'fix_orientation': True,
  'provenance': {'creator': 'QCElemental',
   'version': '0.26.0',
   'routine': 'qcelemental.molparse.from_schema'},
  'extras': {'canonical_isomeric_explicit_hydrogen_mapped_smiles': '[H:1][O:2][H:3]'}},
 'driver': <DriverEnum.energy: 'energy'>,
 'model': {'method': 'hf', 'basis': '6-31G*'},
 'keywords': {},
 'protocols': {'wavefunction': <WavefunctionProtocolEnum.all: 'all'>},
 'extras': {'qcvars': {'CURRENT ENERGY': -76.01072065749648,
   'CURRENT

In [16]:
opt_input_2 = { "molecule" : optmized_mol,
              "driver" : "energy",
              "model" : {"method":"scf","basis":"6-31G*"},
              "protocols":{"wavefunction":"all","stdout":True,"native_files":"all"},
              "keywords":{"scf_properties":["MULLIKEN_CHARGES", "LOWDIN_CHARGES", "DIPOLE", "QUADRUPOLE", "MBIS_CHARGES"]}                               
}
#"GRID_ESP", "GRID_FIELD"
CWD = os.getcwd()

opt_3 = qcengine.compute(opt_input_2, "psi4", local_options={"scratch_directory":CWD,"scratch_messy":True})
opt_3.error

/scratch/ipykernel_27124/248565322.py:10: FutureWarning: Using the `local_options` keyword argument is deprecated in favor of using `task_config`, in version 0.30.0 it will stop working.
  opt_3 = qcengine.compute(opt_input_2, "psi4", local_options={"scratch_directory":CWD,"scratch_messy":True})


In [1]:
import sys
sys.path.append('/mnt/storage/nobackup/nca121/test_jobs/QM_ESP_Psi4/QM_ESP_Psi4/')

from openff.toolkit import Molecule
from openff.recharge.esp import ESPSettings
from openff.recharge.grids import LatticeGridSettings
from openff.recharge.grids import GridSettingsType, GridGenerator
from qcelemental.models.procedures import OptimizationInput, QCInputSpecification
from source.conformers.conformer_gen import Conformers
from openff.units import unit
from qcelemental.models.common_models import Model
from openff.recharge.utilities.molecule import smiles_to_molecule
import qcengine
import os
import numpy as np


CWD = os.getcwd()

#generate water test molecule as openff.toolkit.Molecule
test_mol =  smiles_to_molecule('[H]O[H]')
conformer_list = Conformers.generate(test_mol, generation_type='rdkit')
conformer_list[0]
qc_mol =  test_mol.to_qcschema(conformer=0)

#Setup geometry optimisation
hf_model = Model(method="hf", basis="6-31G*")
spec = QCInputSpecification(model=hf_model, keywords={}, driver="gradient")
opt_spec = OptimizationInput(
            initial_molecule=qc_mol,
            input_specification=spec,
            keywords={"coordsys": "dlc", 
                      "program": "psi4"}                                        
        )

opt = qcengine.compute_procedure(opt_spec, "geometric")
print(f'geometry optimiztion was {opt.error}')
#return optimized molecule
optmized_mol = opt.final_molecule

#Generate grid.dat file for grid_esp and grid_field
grid_settings = LatticeGridSettings(
        type="fcc", spacing=0.5, inner_vdw_scale=1.4, outer_vdw_scale=2.0
    )

grid = GridGenerator.generate(test_mol, optmized_mol.geometry*unit.angstrom, grid_settings)

grid = grid.to(unit.angstrom).m
np.savetxt("grid.dat", grid, delimiter=" ", fmt="%16.10f")
#compute one-electron properties.
opt_input_2 = { "molecule" : optmized_mol,
              "driver" : "energy",
              "model" : {"method":"scf","basis":"6-31G*"},
              "protocols":{"wavefunction":"all","stdout":True,"native_files":"all"},
              "keywords":{"scf_properties":["GRID_ESP", "GRID_FIELD","MULLIKEN_CHARGES", "LOWDIN_CHARGES", "DIPOLE", "QUADRUPOLE", "MBIS_CHARGES"]}                               
              }


opt_2 = qcengine.compute(opt_input_2, "psi4", task_config={"scratch_directory":CWD,"scratch_messy":True})

print(opt_2.dict())

esp = (np.loadtxt("grid_esp.dat").reshape(-1, 1) * unit.hartree / unit.e)

electric_field = (np.loadtxt("grid_field.dat")* unit.hartree/ (unit.e * unit.bohr))

rdkit has generated 10 conformers from a requested 10 conformers
geometry optimiztion was None


/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/nfs/home/nca121/mambaforge/envs/openff/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


{'id': None, 'input_data': {'molecule': {'schema_name': 'qcschema_molecule', 'schema_version': 2, 'validated': True, 'symbols': array(['H', 'O', 'H'], dtype='<U1'), 'geometry': array([[-1.42427673, -0.3639913 ,  0.        ],
       [-0.00142791,  0.72236003,  0.        ],
       [ 1.42570464, -0.35836872,  0.        ]]), 'name': 'H2O', 'molecular_charge': 0.0, 'molecular_multiplicity': 1, 'connectivity': [(0, 1, 1.0), (1, 2, 1.0)], 'fix_com': True, 'fix_orientation': True, 'provenance': {'creator': 'QCElemental', 'version': '0.26.0', 'routine': 'qcelemental.molparse.from_schema'}, 'extras': {'canonical_isomeric_explicit_hydrogen_mapped_smiles': '[H:1][O:2][H:3]'}}, 'driver': 'energy', 'model': {'method': 'scf', 'basis': '6-31G*'}, 'protocols': {'wavefunction': 'all', 'stdout': True, 'native_files': 'all'}, 'keywords': {'scf_properties': ['GRID_ESP', 'GRID_FIELD', 'MULLIKEN_CHARGES', 'LOWDIN_CHARGES', 'DIPOLE', 'QUADRUPOLE', 'MBIS_CHARGES']}, 'provenance': {'cpu': 'Intel(R) Xeon(R) CPU 

FileNotFoundError: grid_esp.dat not found.

In [19]:
from source.optimize.openff_psi4_gen import Psi4Generate
import psi4

esp_settings = ESPSettings(basis="6-31G*", method="hf", grid_settings=grid_settings)

xyz = Psi4Generate.run_calc(test_mol,conformer_list[0],esp_settings,compute_esp=True,compute_field=True, minimize=False)

molecule = psi4.geometry(
"""
3
[H]O[H]
H	-0.81024874	-0.18571238	-0.0
O	-0.00248133	0.36959931	-0.0
H	0.81273007	-0.18388693	0.0
"""
)

molecule.set_molecular_charge(0)
molecule.set_multiplicity(1)

E, wfn = psi4.energy('hf/6-31g*', return_wfn=True, molecule = molecule)
psi4.prop('hf/6-31G*', properties=["GRID_ESP", "GRID_FIELD","MULLIKEN_CHARGES", "LOWDIN_CHARGES", "DIPOLE", "QUADRUPOLE", "MBIS_CHARGES"])
#psi4.core.print_variables()
psi4.core.variables(['MBIS Charges'])

3
[H]O[H]
H	-0.81024874	-0.18571238	-0.0
O	-0.00248133	0.36959931	-0.0
H	0.81273007	-0.18388693	0.0


Scratch directory: /mnt/storage/nobackup/nca121/scratch_dir/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  5, 4, 3
    Auxiliary basis highest AM E, G, H:  6, 5, 4
    Onebody   basis highest AM E, G, H:  6, 5, 4
    Solid Harmonics ordering:            gaussian

*** tstart() called on login02.cluster
*** at Thu Oct 26 15:47:15 2023

   => Loading Basis Set <=

    Name: 6-31G*
    Role: ORBITAL
    Keyword: BASIS
    atoms 1, 3 entry H          line    44 file /mnt/nfs/home/nca121/mambaforge/envs/openff/share/psi4/basis/6-31gs.gbs 
    atoms 2    entry O          line   145 file /mnt/nfs/home/nca121/mambaforge/envs/openff/share/psi4/basis/6-31gs.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
       

{'CURRENT ENERGY': -76.00688623049868,
 'CURRENT REFERENCE ENERGY': -76.00688623049868,
 'DD SOLVATION ENERGY': 0.0,
 'HF KINETIC ENERGY': 75.71208232170089,
 'HF POTENTIAL ENERGY': -151.71896855219958,
 'HF TOTAL ENERGY': -76.0068862304987,
 'HF VIRIAL RATIO': 2.003893749844908,
 'NUCLEAR REPULSION ENERGY': 8.941189202753957,
 'ONE-ELECTRON ENERGY': -122.62733060212095,
 'PCM POLARIZATION ENERGY': 0.0,
 'PE ENERGY': 0.0,
 'SCF ITERATION ENERGY': -76.00688623049868,
 'SCF ITERATIONS': 8.0,
 'SCF TOTAL ENERGY': -76.00688623049868,
 'TWO-ELECTRON ENERGY': 37.679255168868295,
 'HF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ]),
 'HF QUADRUPOLE': array([[-2.78959587e+00, -8.82137120e-04,  0.00000000e+00],
        [-8.82137120e-04, -4.43606969e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -5.41213293e+00]]),
 'SCF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ])}

bol    Alpha    Beta     Spin     Total
       1     H     0.31946  0.31946  0.00000  0.36108
       2     O     4.36116  4.36116  0.00000 -0.72231
       3     H     0.31938  0.31938  0.00000  0.36123

  Total alpha =  5.00000, Total beta =  5.00000, Total charge =  0.00000
  ==> Computing MBIS Charges <==

  Electron Count from Grid (Expected Number): 10.00000 (10.00000)
  Difference:  0.00000

  MBIS Charges: (a.u.)
   Center  Symbol  Z      Pop.       Charge
      1       H    1    0.535424    0.464576
      2       O    8    8.928037   -0.928037
      3       H    1    0.536539    0.463461

  MBIS Dipoles: [e a0]
   Center  Symbol  Z        X           Y           Z
      1       H    1   -0.020186    0.000731   -0.000000
      2       O    8   -0.000896    0.135495   -0.000000
      3       H    1    0.020351    0.000815   -0.000000

  MBIS Quadrupoles: [e a0^2]
   Center  Symbol  Z      XX        XY        XZ        YY        YZ        ZZ
      1       H    1   -0.2445   -0.0019

In [20]:
psi4.core.variables(['MBIS Charges'])

{'CURRENT ENERGY': -76.00688623049868,
 'CURRENT REFERENCE ENERGY': -76.00688623049868,
 'DD SOLVATION ENERGY': 0.0,
 'HF KINETIC ENERGY': 75.71208232170089,
 'HF POTENTIAL ENERGY': -151.71896855219958,
 'HF TOTAL ENERGY': -76.0068862304987,
 'HF VIRIAL RATIO': 2.003893749844908,
 'NUCLEAR REPULSION ENERGY': 8.941189202753957,
 'ONE-ELECTRON ENERGY': -122.62733060212095,
 'PCM POLARIZATION ENERGY': 0.0,
 'PE ENERGY': 0.0,
 'SCF ITERATION ENERGY': -76.00688623049868,
 'SCF ITERATIONS': 8.0,
 'SCF TOTAL ENERGY': -76.00688623049868,
 'TWO-ELECTRON ENERGY': 37.679255168868295,
 'HF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ]),
 'HF QUADRUPOLE': array([[-2.78959587e+00, -8.82137120e-04,  0.00000000e+00],
        [-8.82137120e-04, -4.43606969e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -5.41213293e+00]]),
 'SCF DIPOLE': array([ 0.00408617, -0.83522958,  0.        ])}

[{'element': 'H', 'x': -0.8102487419316984, 'y': -0.18571237832658302, 'z': -0.0}, {'element': 'O', 'x': -0.002481329284462171, 'y': 0.3695993118799398, 'z': -0.0}, {'element': 'H', 'x': 0.8127300712161604, 'y': -0.18388693355335695, 'z': 0.0}]
